# Algoritmos de optimización - Problema 1<br>
Nombre y Apellidos:  Moises Britez <br>
Url: https://github.com/moisesbritez92/MIAR_VIU<br>
Problema:
> 1. Sesiones de doblaje <br>

Descripción del problema:

Se precisa coordinar el doblaje de una película. Los actores del doblaje deben coincidir en las
tomas en las que sus personajes aparecen juntos en las diferentes tomas. Los actores de
doblaje cobran todos la misma cantidad por cada día que deben desplazarse hasta el estudio de
grabación independientemente del número de tomas que se graben. No es posible grabar más
de 6 tomas por día. El objetivo es planificar las sesiones por día de manera que el gasto por los
servicios de los actores de doblaje sea el menor posible.

(*) La respuesta es obligatoria





                                        

In [1]:
import pandas as pd
import numpy as np
#Leemos el archivo Excel
df = pd.read_excel("Datos.xlsx")
#Convertimos el DataFrame a un arreglo de NumPy
arreglo = df[df.columns[1:11]]
arreglo
df

,Toma,1,2,3,4,5,6,7,8,9,10
0,1,1,1,1,1,1,0,0,0,0,0
1,2,0,0,1,1,1,0,0,0,0,0
2,3,0,1,0,0,1,0,1,0,0,0
3,4,1,1,0,0,0,0,1,1,0,0
4,5,0,1,0,1,0,0,0,1,0,0
5,6,1,1,0,1,1,0,0,0,0,0
6,7,1,1,0,1,1,0,0,0,0,0
7,8,1,1,0,0,0,1,0,0,0,0
8,9,1,1,0,1,0,0,0,0,0,0
9,10,1,1,0,0,0,1,0,0,1,0


Agrupamos tomas en bloques de 6 tomas por día (máximo).

Para cada día, computamos cuántos actores participan en al menos una de esas tomas.

Sumamos todos los días por los que cada actor debe cobrar.

In [2]:
# Número máximo de tomas por día
MAX_TOMAS_POR_DIA = 6
participacion = arreglo
# Agrupar las tomas de 6 en 6
dias = [participacion[i:i+MAX_TOMAS_POR_DIA] for i in range(0, len(participacion), MAX_TOMAS_POR_DIA)]

# Calcular cuántos días trabaja cada actor
actores_dias = np.zeros(participacion.shape[1], dtype=int)

for dia in dias:
    # Para cada actor, si participa en al menos una toma del día, se le paga ese día
    actores_en_dia = np.any(dia, axis=0)  # vector booleano de tamaño 10
    actores_dias += actores_en_dia.astype(int)

# Costo total: suma de días trabajados por todos los actores
costo_total = actores_dias.sum()
print(f"Costo total: {costo_total} días trabajados")
# Mostrar resumen de resultados
resumen = pd.DataFrame({
    "Actor": list(range(1, 11)),
    "Días trabajados": actores_dias
})

resumen

Costo total: 38 días trabajados


,Actor,Días trabajados
1,1,5
2,2,5
3,3,5
4,4,5
5,5,5
6,6,4
7,7,2
8,8,3
9,9,2
10,10,2


In [3]:
import numpy as np
import pandas as pd

# Cargar los datos estructurados desde un archivo Excel previamente limpio
df_nuevo = pd.read_excel("Datos.xlsx")

# Eliminamos la columna de identificación de tomas ("Toma")
participacion = df_nuevo.drop(columns=["Toma"]).values  # Matriz 30 x 10

# Parámetro: número máximo de tomas por día
MAX_TOMAS_POR_DIA = 6

# Inicializamos dos listas:
dias = []                # Cada elemento será una lista de índices de tomas para ese día
actores_por_dia = []     # Cada elemento es un set de actores presentes en ese día

# Recorremos cada toma por su índice y su vector de participación
for idx, toma in enumerate(participacion):
    colocada = False
    actores_toma = set(np.nonzero(toma)[0])  # Actores que participan en esta toma

    # Intentamos colocar esta toma en algún día ya existente
    for i, dia in enumerate(dias):
        if len(dia) < MAX_TOMAS_POR_DIA:
            # Calculamos qué actores nuevos aparecerían si agregamos esta toma
            nuevos_actores = actores_toma - actores_por_dia[i]

            # Si no agregamos demasiados actores nuevos, colocamos la toma en este día
            if len(nuevos_actores) <= 4:  # este umbral se puede ajustar
                dias[i].append(idx)
                actores_por_dia[i].update(actores_toma)
                colocada = True
                break

    # Si no fue posible ubicarla en ningún día, se crea un nuevo día
    if not colocada:
        dias.append([idx])
        actores_por_dia.append(actores_toma.copy())

# Inicializamos el vector de días trabajados por cada actor
dias_trabajados_heur = np.zeros(participacion.shape[1], dtype=int)

# Recorremos la lista de actores por día, y sumamos un día a cada actor participante
for actores_dia in actores_por_dia:
    for actor in actores_dia:
        dias_trabajados_heur[actor] += 1

# Mostramos los resultados finales
resumen = pd.DataFrame({
    "Actor": list(range(1, 11)),
    "Días trabajados (heurística)": dias_trabajados_heur
})

print(resumen)

# Costo total: suma de todos los días pagados
costo_total = dias_trabajados_heur.sum()
print(f"\nCosto total de días trabajados: {costo_total}")



   Actor  Días trabajados (heurística)
0      1                             5
1      2                             5
2      3                             5
3      4                             5
4      5                             5
5      6                             4
6      7                             2
7      8                             3
8      9                             2
9     10                             2

Costo total de días trabajados: 38


(*)¿Cuantas posibilidades hay sin tener en cuenta las restricciones?<br>



¿Cuantas posibilidades hay teniendo en cuenta todas las restricciones.




Respuesta

In [4]:
import math

total_posibilidades_sin_restricciones = math.factorial(30) // (math.factorial(6)**5 * math.factorial(5))
total_posibilidades_sin_restricciones


11423951396577720

Modelo para el espacio de soluciones<br>
(*) ¿Cual es la estructura de datos que mejor se adapta al problema? Argumentalo.(Es posible que hayas elegido una al principio y veas la necesidad de cambiar, arguentalo)


Respuesta

Según el modelo para el espacio de soluciones<br>
(*)¿Cual es la función objetivo?

(*)¿Es un problema de maximización o minimización?

Respuesta

Diseña un algoritmo para resolver el problema por fuerza bruta

Respuesta

Calcula la complejidad del algoritmo por fuerza bruta

Respuesta

(*)Diseña un algoritmo que mejore la complejidad del algortimo por fuerza bruta. Argumenta porque crees que mejora el algoritmo por fuerza bruta

Respuesta

(*)Calcula la complejidad del algoritmo

Respuesta

Según el problema (y tenga sentido), diseña un juego de datos de entrada aleatorios

Respuesta

Aplica el algoritmo al juego de datos generado

Respuesta

Enumera las referencias que has utilizado(si ha sido necesario) para llevar a cabo el trabajo

Respuesta

Describe brevemente las lineas de como crees que es posible avanzar en el estudio del problema. Ten en cuenta incluso posibles variaciones del problema y/o variaciones al alza del tamaño

Respuesta